<h1 align='center'> 영상처리 프로그래밍 실습 7</h1>

<h6 align='right'> 2025. 5. 8. </h6>

<div class="alert alert-block alert-info">
    
- 파일 이름에서 00000000을 자신의 학번으로, name을 자신의 이름으로 수정하세요.

- 다음 줄에 자신의 이름, 학번, 학과(전공)을 적으세요.

* 이름:   &nbsp;&nbsp;          학번:    &nbsp;&nbsp;         학과(전공):
    
</div>

- JupyterLab 문서의 최신 버전은 [JupyterLab Documentation](https://jupyterlab.readthedocs.io/en/stable/index.html#/)을  참고하라

- Markdown은 [Markdown Guide](https://www.markdownguide.org/)를 참고하라.
- [Markdown Cheat Sheet](https://www.markdownguide.org/cheat-sheet/)

* 제출 마감: 5월 14일 (수) 오후 10:00까지 최종본을 SmartLEAD제출


In [1]:
import cv2
import matplotlib.pyplot as plt

import numpy as np
print("OpenCV version", cv2.__version__)
print("NumPy version", np.__version__)

OpenCV version 4.10.0
NumPy version 1.26.4


## 문제 1.

지난 주에 작성했던 PyQt5를 이용한 GUI에 몇 가지 기능을 추가했던 프로그램을 실행한 후에 윈도우의 크기를 조정해 보자. 윈도우의 크기를 조정한 후에 영상의 크기도 윈도우의 크기에 비례해서 조정하려면 어떤 기능을 추가해야 하는지 조사해서 다시 작성하라.

In [2]:
import sys
from PyQt5.QtWidgets import (
    QApplication, 
    QMainWindow, 
    QLabel,
    QMenuBar, QAction,
    QFileDialog, QHBoxLayout, QWidget
)
from PyQt5.QtCore import Qt 
from PyQt5.QtGui import QPixmap, QImage
import image_tools  # 사용자 정의 모듈 
import numpy as np

class BasicViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("단순 이미지 뷰어")
        self.setGeometry(100, 100, 1000, 600)

        # QLabel 두 개: 원본 / 변환
        self.original_label = QLabel("원본 이미지를 열어보세요")
        self.original_label.setAlignment(Qt.AlignCenter)

        self.processed_label = QLabel("변환된 이미지가 여기에 표시됩니다")
        self.processed_label.setAlignment(Qt.AlignCenter)

        # 수평 레이아웃
        self.central_widget = QWidget()
        self.hbox = QHBoxLayout(self.central_widget)
        self.hbox.addWidget(self.original_label)
        self.hbox.addWidget(self.processed_label)

        self.setCentralWidget(self.central_widget)

        self.current_image = None  # 원본 이미지 저장용 (NumPy)

        self.create_menu()

    def create_menu(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        open_action = QAction("열기", self)
        open_action.triggered.connect(self.open_image_dialog)
        file_menu.addAction(open_action)

        exit_action = QAction("종료", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        # 변환 메뉴
        transform_menu = menubar.addMenu("변환")
        log_action = QAction("로그 변환", self)
        log_action.triggered.connect(self.apply_log_transform)
        transform_menu.addAction(log_action)

    def open_image_dialog(self):
        filename, _ = QFileDialog.getOpenFileName(
            self, "이미지 열기", "", "Image Files (*.png *.jpg *.bmp *.jpeg *.tif *.tiff)"
        )
        if filename:
            image = image_tools.load_image(filename)  # RGB 형태로 반환
            self.current_image = image
            pixmap = self.numpy_to_qpixmap(image)
            self.original_label.setPixmap(pixmap.scaled(
                self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
            self.processed_label.clear()

    def apply_log_transform(self):
        def log_transformation(image, f_max=255):
            C = 255 / np.log(1 + f_max)
            return (C * np.log(1. + image)).round().clip(0, 255).astype(np.uint8)

        if self.current_image is None:
            return

        transformed = log_transformation(self.current_image, f_max=self.current_image.max())
        pixmap = self.numpy_to_qpixmap(transformed)
        self.processed_label.setPixmap(pixmap.scaled(
            self.processed_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
        ))

    def numpy_to_qpixmap(self, image):
        h, w, ch = image.shape
        bytes_per_line = ch * w
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        return QPixmap.fromImage(qimage)

app = QApplication.instance()
if app is None:
    app = QApplication([])

viewer = BasicViewer()
viewer.show()

try:
    sys.exit(app.exec_())
except SystemExit:
    pass


#### 수정 후 프로그램
- 원 영상과 처리된 영상을 표시하는 두 개의 QLabel의 setSizePolicy 함수에 QSizePolicy.Ignored 인자를 전달
- resizeEvent 처리 함수를 추가하여 윈도우 크기가 변할 때마다 다시 영상의 크기를 조정하여 표시

### 문제 2. 

다음을 만족하는 함수 moving_average_filtering(filename)를 완성하라.

- filename으로 전달받은 파일이 존재하는지 조사해서 존재하지 않으면 오류 메시지를 콘솔에 출력하고 return 한다.
- 파일이 존재하면
  - 파일을 읽고, 원본 영상과 이동 평균 필터링된 영상을 나란히 화면에 표시한다.
  - 이동 평균 필터의 커널의 크기를 지정할 수 있는 trackBar interface를 제공한다. 단, trackBar로 지정하는 값의 범위는 1부터 51까지로 하되, 짝수일 경우 그보다 1 작은 홀수로 커널의 크기를 지정한다.
  - 지정된 커널의 크기를 이용해서 원 영상에 이동 평균 필터링이 적용된 영상을 구한다.

### 2.1 OpenCV GUI 기반

### 2.2 PyQt 기반

## 문제 3.
강의에서 설명한 여러 가지 필터를 필터 메뉴에 추가하라.

### 문제 4.

다음 URL에서 'Lincoln.jpg',  'Lincoln_wave_1.png', 'Lincoln_wave_2.png' 파일을 로컬 드라이브에 다운로드하라.

https://drive.google.com/drive/u/0/folders/1zbjtkf9nHy9VniuLI4wHilbrN_JBvhYi

로컬 드라이브에서 'Lincoln.jpg' 파일을 읽고, 'Lincoln_wave_1.png' 또는 'Lincoln_wave_2.png'와 같은 그림을 화면에 표시하는 프로그램을 작성하라.

참고 사이트: https://mymodernmet.com/tyler-foust-one-line-drawing-portraits/